Global setup



In [28]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8(?) for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2

In [29]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [30]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [31]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.765852644390328

Expected number of repetitions:

In [32]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

1.70494183535004

Alice generates a signing key

In [33]:
#A = random_matrix(ZZ, 4, 4, x=0, y=q)
A = matrix(4,4, 
[[ 4, 14,  4, 14],
[27, 28,  8, 40],
[ 9,  5,  8, 19],
[ 8,  9, 14,  3]])
A

[ 4 14  4 14]
[27 28  8 40]
[ 9  5  8 19]
[ 8  9 14  3]

In [34]:
latex(A)

\left(\begin{array}{rrrr}
4 & 14 & 4 & 14 \\
27 & 28 & 8 & 40 \\
9 & 5 & 8 & 19 \\
8 & 9 & 14 & 3
\end{array}\right)

In [35]:
#S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S1 = matrix(ZZ, 
[[-1,  0, -1,  1],
[ 0,  1,  0,  1],
[ 1, -1,  0,  1],
[-1,  0,  0,  0]])
S1

[-1  0 -1  1]
[ 0  1  0  1]
[ 1 -1  0  1]
[-1  0  0  0]

In [36]:
#S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S2 = matrix(ZZ, 
[[ 0, -1, -1,  0],
[ 1, -1,  0, -1],
[ 0, -1,  1,  1],
[ 0,  1,  0, -1]])
S2

[ 0 -1 -1  0]
[ 1 -1  0 -1]
[ 0 -1  1  1]
[ 0  1  0 -1]

$(S_1, S_2)$ is the private key


In [37]:
T = matrix(((A*S1 + S2) % q))
T

[27  9 36 22]
[24 19 14 21]
[21 37 33 23]
[ 3 37 33 30]

In [38]:
latex(T)

\left(\begin{array}{rrrr}
27 & 9 & 36 & 22 \\
24 & 19 & 14 & 21 \\
21 & 37 & 33 & 23 \\
3 & 37 & 33 & 30
\end{array}\right)

$(A,T)$ is the public key


Alice signs the message



In [39]:
M = 'HOLA'
Mlist = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
Mlist

[8, 15, 12, 1]

In [40]:
#y1 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y1 = matrix(k, 1, [-3, 0, -1, 14])
y1

[-3]
[ 0]
[-1]
[14]

In [41]:
#y2 = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y2 = matrix(k, 1, [0, -12, 9, -2])
y2

[  0]
[-12]
[  9]
[ -2]

In [42]:
w1 = matrix(((A*y1+y2) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1

[16]
[ 8]
[-6]
[ 2]

Start the "non-cryptographic hash/XOF (combined?)"

In [43]:
Mlist

[8, 15, 12, 1]

In [44]:
w1list = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1list

[16, 8, -6, 2]

In [45]:
vector(Mlist).dot_product(vector(w1list)) % (2**6)

50

In [79]:
def Dbox(list1, list2, itemlen):
    newitem = vector(list1).dot_product(vector(list2)) % (2**itemlen) # make sure all bits of input are used
    out = format(newitem, f'0{itemlen}b')
    return out

In [80]:
c_hash = Dbox(Mlist, w1list, 6)
c_hash

'110010'

Use Dbox to sample a sparse vector

In [81]:
clenbits = log(k,2)
clenbits

2

In [82]:
def sample_in_ball(digest):  # digest should be at least ****
    n = 2**clenbits  # variable name conflicts!  This n is local
    c = [0] * n
    k = clenbits # This k is local
    for i in range(n-tau, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+tau-n], 2)
        k = k+1
    return c

In [83]:
c = matrix(k,1,sample_in_ball(c_hash))
c

[ 0]
[ 0]
[-1]
[ 0]

In [85]:
z1 = matrix(y1 + S1*c); z1

[-2]
[ 0]
[-1]
[14]

Is $\|\mathbf{z_1}\|_\infty \geq \gamma_1-\beta$?

In [23]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_1}$

In [86]:
z2 = matrix(y2 + S2*c); z2

[  1]
[-12]
[  8]
[ -2]

Is $\|\mathbf{z_2}\|_\infty \geq \gamma_1-\beta$?

In [25]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y_2}$

If not, $(\mathbf{z_1}, \mathbf{z_2}, \mathbf{c})$ is the signature


In [87]:
(z1, z2, c)

(
[-2]  [  1]  [ 0]
[ 0]  [-12]  [ 0]
[-1]  [  8]  [-1]
[14], [ -2], [ 0]
)

Bob verifies the signature

Is $\|\mathbf{z_i}\|_\infty < \gamma_1-\beta$?

In [88]:
z1

[-2]
[ 0]
[-1]
[14]

In [89]:
z2

[  1]
[-12]
[  8]
[ -2]

In [90]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [92]:
w1prime = matrix(((A*z1+z2-T*c) % q)).apply_map(lambda r: Mod(r,q).lift_centered())
w1prime

[16]
[ 8]
[-6]
[ 2]

In [93]:
w1primelist = [c for row in w1 for c in row]  #flatten the matrix down to coefficients
w1primelist

[16, 8, -6, 2]

In [94]:
cprime_hash = Dbox(Mlist, w1primelist, 6)
cprime_hash

'110010'

In [95]:
cprime = matrix(k,1,sample_in_ball(cprime_hash))
cprime

[ 0]
[ 0]
[-1]
[ 0]

In [35]:
c == cprime

True

Yes.  The signature is accepted.